[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1wfaaIZfBettCzyZqvYn6LeCOgORgq-Td?usp=sharing)

# Testing Vision models with a Single API

It is imperative to test different models for your use case to ensure optimal performance and accuracy. However, this process can be cumbersome, as it often involves downloading multiple packages and navigating various APIs. The complexity and effort required can be a significant barrier to effective evaluation.

Portkey simplifies this process significantly. As an open-source [**AI Gateway**](https://github.com/Portkey-AI/gateway), Portkey provides a streamlined solution to manage access to over 250 models through a single, unified API. This not only saves time and reduces complexity but also offers valuable insights into cost, performance, and accuracy metrics.

With Portkey, you can efficiently compare and evaluate a wide range of LLMs, ensuring you choose the best model for your needs.

In this notebook, we illustrate how to run different vision models with a single unified API using Portkey.

### Quickstart

In [1]:
!pip install -qU portkey-ai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


To start, get your Portkey API key by signing up https://app.portkey.ai/. (Go to "Settings" -> "API Keys" -> Create an API key with apporpriate scope)

### Using Portkey to run multiple LLMs

In [3]:
## List of Popular vision models
vision_models = [
    ["gpt-4o", "openai"],
    ["gpt-4-turbo", "openai"],
    ["claude-3-opus-20240229", "anthropic"],
    ["claude-3-sonnet-20240229", "anthropic"],
    ["claude-3-haiku-20240307", "anthropic"],
]

## Replace this with your own virtual keys
virtual_keys = {
    "openai": "gpt3-8070a6",                     # Grab from https://platform.openai.com/
    "anthropic": "anthropic-9e8db9",               # Grab from https://console.anthropic.com/
}

In [2]:
from google.colab import userdata

PORTKEY_API_KEY = userdata.get('PORTKEY_API_KEY')

In [8]:
from openai import OpenAI
from portkey_ai import PORTKEY_GATEWAY_URL, createHeaders
import json

# Using the OpenAI SDK to run multiple LLMs in a loop

def compare_vision_models(prompt, img_url):
    outputs = {}
    img_url = img_url


    for model, provider in vision_models:
        portkey = OpenAI(
            api_key = "dummy_key",
            base_url = PORTKEY_GATEWAY_URL,
            default_headers = createHeaders(
                api_key=PORTKEY_API_KEY,
                virtual_key = virtual_keys[provider]
            )
        )

        response = portkey.chat.completions.create(
            messages=[
          {"role": "user", "content": [
              {"type": "text", "text": prompt},
              {"type": "image_url", "image_url": {
                  "url": img_url}
              }
          ]}
          ],
            model=model,
            max_tokens=256
        )

        content = response.choices[0]
        outputs[model] =content.message.content

    return outputs

In [6]:
from tabulate import tabulate

def print_model_outputs(prompt, img_url):
    outputs = compare_vision_models(prompt, img_url)

    table_data = []
    for model, output in outputs.items():
        table_data.append([model, output.strip()])

    headers = ["Model", "Output"]
    table = tabulate(table_data, headers, tablefmt="grid")
    print(table)
    print()

In [16]:
print_model_outputs(prompt = "Solve the question",
                    img_url = "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAWkAAACMCAMAAACJW6j5AAAAyVBMVEX///93GoB+Kobl1+bu5e+BNYl9K4V7JoN6I4N+Lob79/t5IIL9+/3y7vPj2uV7JIOeW6F0JX6UWZvfz+GFO4zcyt7Ns9DCo8a1j7rn2+lxAHqOTJWug7OicKiIQZD09PSeaaS0tLaaYqDXw9moeq1yAHu7mL/HqsrMzM2yirfPuNK0jrmNSpTUvdbV1tbq6uvBwcLIuMygh6eBW4tfLW1dPWtfH2uKdJSTf5yvnLZ6M4NXF2VvVny4q71yTX6sra6enqGTlJaAgoSTor3kAAAJH0lEQVR4nO2dC3ubOBaGJYwkhGDVAQzmam6BmODL7GV2d2Zn0+7//1ErnCZxHbtxW7uArfdpYhtIopwqn875kGQAJBKJRCK5GvRd+m7MFWMsPuzSAOAtFgvPBHrdhX06FZ8mBTBqE3SfjEoFfvWwNPpu9/gwAyrAhBPcPVrARAQrMNGnQSXOolBcY1FgB56I+F0tHn0GYRD13e7xYUKEsFPOvMqiCFHLMJ3UNN2gAZbigwguxTWW8znSsJ4G0wdWmAto9t3w0SEijWe//vVvf//HEn2OdAzA8q4BUxhPWdxd8xppT4/BA3TFl/Xd7vEhIk2a3zFFv/0zps+RtnAuTkUEaWp3zU6kxSuVQmvZb6NHybZPp51w/Ct/iXQOHTESmi3zttd8GWmgewiWvTZ6lIhI03yBEfrt3w1+UQ8DteLcjJtgKUL+wQF+MAOgeIq0OF0Gdq+tHiPdiEg90ad//+M/LqE52EYaJEjkeA+aCRwF6KwFJrN0UAZdzuekBnCDuu+Gj44u0sQLxWiI/vjToSLLc1CSQNilcQuoggoKxBA46x6t7ivKQEuYIofEb2XbpyMxGAr9+DMR6gHKMAzzbZctUtGxayt3uxdNnldP9UodJqnfY5NHShdplLgEUfLfJekifQKyQvwOtpEmLkV5tXS6EdHIHler7an1+vWy+er5Sfbz23gdbCNNo6VbpXhbuYB1Nv+YzbN7sLoH99l8c39/v8lW4mOdZQbIPs7Fs00277vho2MbaUQTgin6HOnNfLN5zDIR6fmnzf3mU2ZkqywTHysR3/9l2eP8cSP+FwaGvhz4IK2Hv+zAHjqhMFardbYC60fRm9ebjeja69Xqfr3OPs3X9/PNx2z1cXB9Wreg4/bdiK8z/8suXx3r7p868mr7b1gYOcaY5TIlujgxgQ85xMqDvJdxWWLCZgC4CiFO03dbrpoF17aWlx5Byi0pIRdjwcizt+iHDMOFrKwugwdJ/hpblyoESwm5BB7E+W4nNiNCoZSQ89MwbO3VLLbFMZzJLOS8NBCHb4tD15FZyJmZctqqB46bEcdMervnw2bUORTo7lROqVJJCTkPPqJ0cuzkFCPMW3kX/xyIQKNjAqHPFJo8cKxJCflx1JYi5B6sUow64cSZgonFMJZZyA9itpQolLfeG6FWvUSkedE2JamxyEKkhPwIukVh41sQE2x59ku31W3PEoeg9azfZgmpZh0ZNiXvY1gYdjfw7Rhq3UxZZMUCC3WTZjUY746Tk4TR7haH5LvICauenplNTjSNky1c05S02RfmmmFCpYR8F0+G9At+4UVdn4684mCqoZeMwlxKyLcTaeQb58pPQlE0VtJO/UYeXg3pZ4qqy0EMr+NwMVOLQiYpLt+4a6JiON7rnWXgKNAHesAgvHPBJI7fToY1IwXDWErI6ewb0gIbRrpKcqAGtar6pkGLKTxQr0wtDXPphZyKSCTy/WBVgcgsUgam0FZFp1UVAPihodGoRRooJeQ0lhC3bwzpuptDH2qqqyUBKw3g1C4/PPqZJcNaLL2Q91keNKRVBS1SQtRamdmpqGjURXRUj+2QYSol5D1shSqHYmhbSpxrajff2H9acPYVaoUQaad+HZtSfijQRrcOOySgKAygsneXO3VeCE9lFnIcFVF4MFn2Yao3MALpnatHwQkzIe1WFOjVudt3NagOhUfWjC3uYJDowCcBvNtPAQ9Ta5g4Mgs5iJlQeDQ0trsVXmPpnrp+z9x6IQOfdN0LZovZeYcxuyUYSi9kHyPH7OyLID0svZB9jJxo3vm/rRpzzKQXsktJ+Oz9q76Dacix4slC5plS44sLfWvDQ0QLpYQ8ccCQPiNqqWGtlBICLh1o8GSnUk9mIQcM6XNjeBRr4fSyP2TweAyftpz9h1C7eSG3nYU0nCY/pZKzHX7TErLUKP1ZOVgFMWlvVUKOGdKXwYwhhfFNeiETftiQvhjThGB4gxLiI6ocnYp+ISqKeXJrEqIiqv38TczUmFNY3pSEqC1WeqmSi5BjcgE/a6joLdZ6uq9qVLdUyOgWJv3tyqfG7Ga8kJywXv+Ai5YTdAtZSEx4z/eu9UrB3Lp6CYn5xQzp01FjUTReuYRE2oV90hMpks4L6bsVF2TGSNp3G54wKnbNhUylbbcaHwZqCimLr/NGYw1x0ncbdikQwdo1bgPtMhoOrAtVHLP26jY3Lxh1Bmc6qKl2dV5IodBkiHnVMiEY11dUyEzo8Z0k+kWvKGbW1UjIBFH0sw3pk/FjDbMrkRDVwXjIueuyFYXMNWQhaoLpkAPdSYhI+Mbvhegh5oPfek0UMphH45YQEWgy8M26txTdzkOjlpAcs3Gs8dEr0a1HnIW87o0yfDovRBtrFlISNqa3tywowcooJaTkfEyBBsCYieQ6GWzuf5SZNhRD+nT8fIReSMXIe4u9h8jSGZuE1BxbA/NJT0OfKaPyQmptpIEW+KmQ68VIJGSJcThEn/REmlYUMmOouEBB8CAN6QOY06Jrqdq9m9Nrk7t5ISwffhZiU+oM05B+g+0EgdIAkAaCVLxuPqtGJyFk6F6IjykdSaCNkNoTBAForcZ1C1BZi5d1IU1CCBp0FqIiSscydJuxB0AemABu11UbyAT5y1xYc8apZg1XQkxMtTGZvb5LU6AHuYNKHbRT4Ow0fooRJdFAcyg9oWfesuPC5AF3wSRQvByWYoSBuwbCDFOHKGiYWUhC4TAbdgxzEgYq8E1gJGhvADQRjsACUi0cnoQYFu53hvQ3onp2t7K60VVD5B/7a/dqDrvdJy0mevrQJGRMhnSHGuQGSO/8OqiAr1h7Z0P85Nw0DhnaO6yVRBvZlvIldBwYA53CluzvveVj9vmIvlDwoN5hLeLauAxpIXdNHHfjilml0b4Ye/x1I1U/5xguhiIhD5wPYir6uUi/sH3drpAZhoRUg5mKfh70ln9RIZoLPgwvxOMkHcpf11kQMr1XgU1yhknvdqrL6GgN6cNM2Ns93JuEkqTfeqHhyCn8yRXhVwS9PTj9gBDv893mVYgQospVQRHChw6KXxT2Z76r8Lbo8TaHrt4SfQ+KEolEIpFIJBKJRCKRSCQSiUQikUgkEolEIpFIJBKJRPLD/B8tlaiKrWBL7AAAAABJRU5ErkJggg=="
                    )

+--------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Model                    | Output                                                                                                                                                                                  |
+==========================+=========================================================================================================================================================================================+
| gpt-4o                   | To find the unknown angle in a triangle, we can use the fact that the sum of the internal angles of a triangle is always 180°.                                                          |
|                          |                                                                                                                